In [7]:
#:-*- coding: utf-8 -*-

"""
Double moving average line strategy test
Backtest the solid trading engine
@author Matt Yin
@version: 0.5.2
"""
#import
import os
import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from xquant import SignalEvent, Strategy, CSVDataHandler, SimulatedExecutionHandler, BasicPortfolio, Backtest
from xquant.finance.perform import perform_metrics
pd.set_option('display.max_colwidth',None)


py38_site_packages/event.py
event.py - Event
event.py - TickEvent
event.py - BarEvent
event.py - SignalEvent
event.py - OrderEvent
event.py - FillEvent
py38_site_packages/data. py
data.py - DataHandler
data.py - CSVDataHandler
data.py - HDFSDataHandler
py38_site _packages/strategy.py
strategy.py - Strategy
strategy.py - BuyAndHoldStrategy
strategy.py - MovingAverageCrossStrategy
py38_site_packages/portfolio.py
portfolio.py - Portfolio
portfolio.py - BasicPortfolio
py38_site_packages/execution.py
py38_site_packages/commission. py
commission.py - Commission
commission.py - ZeroCommission
commission.py - PerShareCommission
commission.py - PerMoneyCommission
commission.py - PerTradeCommission

execution.py - ExecutionHandler
execution.py - SimulatedExecutionHandler
py38_site_packages/backtest.py
backtest.py - Backtest
py38_site_packages/perform.py

*More features to add/correct*

. Draw K line and signal to facilitate the verification of debugging signal availability errors
· Changed the position size to market value and evenly distributed (solved)
· Fix detail _blotter's IX problem
· Add a benchmark comparison function
· Show order_price, making sure that slippages are taken into account
· Specific calculation formula of commission fee
· Add the ability to use stock prices as a strategy alone
· Figure out the order relationship between generate_naive_order and the entire Backtest organization
· Add real conditional restriction scenarios for CASH
· Confirm whether to enter at the same day's close price
· Eliminate starting time backtest problems
· Comment out all unnecessary prints to increase the recovery speed
·2021/12/8 UDOW+TQQQ backtest results inexplicably increased by 6%
· The backtest result of multiple varieties on the cloud server is wrong. It is suspected that the global variable list is external

In [ ]:
class BuyAndHoldStrategy(Strategy):
    """
    The simplest example: a long position on all stocks
    Used for: Test code: as a benchmark
    """

    def __init__(self,bars,events):
        # Initialize a buy-and-hold strategy
        # parameters:
        # BARS: The data of a DataHandler object, which has many properties and methods. See the Data module
        # Events: The Event object  
        self.bars =bars
        self.symbol_list = self.bars.symbol_list
        self.events =events
        self.strat_name = "BuyAndHold"
        self.bought = self._calculate_initial_bought()
        #self.signal_vals = {}

    def _calculate_initial_bought(self):
        #Add key to the bought dictionary and set the value of all stocks to False, meaning not held
        bought = {}
        for s in self.symbol_list:
            bought[s] = False
        return bought

    def calculate_signals(self, event):
        # This strategy produces only one signal for each stock, which is buy, and no exit signal
        # Buy and hold from the time the strategy was initialized
        # parameters:
        # Event: MarketEvent object 
        if event.type == 'BAR':
            for s in self.symbol_list:
                # Add a ticker to signal df for every ticker you load
                ticker_list_for_signal_df.append(s)
                bar = self.bars.get_latest_bar(s)
                bars = self.bars.get_latest_bars(s, N=5)
                #Add it once for every loading time
                datetime_list_for_signal_df.append(bar.Date)
                #Add the strategy name once for each loaded policy name
                strategy_list_for_signal_df.append(self.strat_name)

                #The original code
                #if bars is not None and bars != [] and self.bought[s] is False:
                #   # format(Symbol,Datetime,Type = Long, SHORT or EXIT)
                #   signal = SignalEvent(bars[0][0], bars[0][1], 'LONG')
                #   self.events.put(signal)
                #   self.bought[s] = True


                if not self.bought[s]:
                    signal = SignalEvent(bar.symbol, bar.Date, 'LONG')
                    self.events.put(signal)
                    self.bought[s] = True
                    signal_list_for)signal_df.append("No position, so buy")

                else:
                    signal_list_for_signal_df.append("Have a position")
    


In [ ]:
class MovingAverageCrossStrategy(Strategy)
    #Double moving average line strategy

    def __init__(self, bars, events, long_window=10, short_window=5):
        # Initializes the doublemoving average strategy
    # parameters:
    # Bars: DataHandler objects
    # Events: Event queue object
    #long_window: the length of the long-term moving average
    #short_window: length of short-term moving average 
    self.bars = bars
    self.symbol_list = self.bars.symbol_list
    self.events = events
    self.long_window = long_window
    self.short_window = short_window
    self.strat_name = "double moving average strategy"
    self.bought = self._calculate_initial_bought()

def _calculate_initial_bought(self):
    # Adds the symbol's holding status to the dictionary, initialized as unheld.
    bought = {}
    for s in self.symbol_list:
        bought[s] = False
    return bought

def calculate_signals(self, event):
    # When the short-term moving average (e.g., 5-day line) above the long-term moving average (e.g., 10-day line), buy; Instead, sell.

    if event.type == 'BAR':
        for s in self.symbol_list:

            # Add a ticker to signal df for every ticker you load
            ticker_list_for_signal_df.append(s)
            
            bar = self.bars.get_latest_bar(s)
                
            #Add it once for every loading time
            datetime_list_for_signal_df.append(bar.Date)
               
            #Add the strategy name once for each loaded policy name
            strategy_list_for_signal_df.append(self.strat_name)

            if bar is None or bar == []
                print("bar is None or bar ==[], no signal")
               # signal_list_for_signal)df.append("no signal")
                continue

            bars =self.bars.get_latest_bars(s,N=self.long_window)
            #If the cycle length of the channel prints the length of the long cycle before proceeding down
            if len(bars) >= self.long_window:
                #Take the dataframe of Bars length only
                df = pd.DataFrame(bar, columns= ['symbol','datetime','open','high','low','close','volume'])
                df['MA_l'] = df['close'].rolling(self.long_window, min_periods=1).mean()#long term 10
                df['MA_s'] = df['close'].rolling(self.short_window, min_periods=1).mean()#short term 5

                print("date:", bar.Date)
                print("ticker: ", s)
                print("MA_s:" df['MA_s'].iloc[-1])
                print("MA_l:" df['MA_l'].iloc[-1])
                print("pre_MA_s:", df['MA_s'].iloc[-2])
                print("pre_MA_l:", df['MA_l'].iloc[-2])


                #Open position signal: if the short line on the long line (strict into)
                if df['MA_l''].iloc[-1] < df['MA_s'].iloc[-1] and df['MA_l''].iloc[-2] > df['MA_s'].iloc[-2]
                    #Add a ticker to signal df for every ticker you load
                    print("Long signal")
                    #signal_list_for_signal_df.append("Long signal")

                    #if no positions,buy:
                    if not self.bought[s]:
                        signal = SignalEvent(bar.symbol, bar.Date, 'LONG')
                        self.events.put(signal)
                        self.bought[s] = True
                        #print("LONG")
                        print("Already buy")

                    #if have positions
                    else:
                        print("Already have Long signal")
                        #signal_list_for_signal_df.append("Already have Long signal")


                #Open position signal: if the short line is below the long line, it is important to add the exit condition as long as the MA_s is below the MA_l for the day (wide exit).
                elif df['MA_l'].iloc[-1] > df['MA_s'].iloc[-1] and df['MA_l'].iloc[-2] < df['MA_s'].iloc[-2] or \
                    df['MA_s'].iloc[-1] < df['MA_l'].iloc[-1]:
                    #Every time a full position signal is generated, add one to the signal df
                    print("Long clearance")
                    #signal_list_for_signal_df.append("Long clearance")
                    
                    #If there is a position, close the position:
                    if self.bought[s]:
                        signal = SignalEvent(bar.symbol, bar.Date, 'EXIT')
                        self.events.put(signal)
                        self.bought[s] =False
                        #print("EXIT")
                        print("The long position has been cleared")

                    #If there is no position, stay by.
                    else:
                        print("Open position signal, but no position")
                        #signal_list_for_signal_df.append("Open position signal, but no position")


                #If there's no signal
                else:
                    print("market no signal")
                    #signal_list_for_signal_df.append("market no signal")
            else:
                print("len(bars) < self.long_window, no signal")
                #signal_list_for_signal_df.append("len(bars) < self.long_window, no signal")

            print("===========================")
        # else:
        # print("no signal")
        # signal_list_for_signal_df.append("no signal")
                        

        




            

In [ ]:
#Continue
class MovingAverageStrategy(Strategy):